In [2]:
import dataAcquisition as DA
import params
import cloud
import backtestSimulator

In [2]:
etfDailyTradingValue = cloud.getApplicableEtfs()

EXI 2140994.438007
MXI 2707252.2237299997
JXI 1250162.108088
KXI 2304863.438644
CVY 1435233.0750240001
WMCR 104652.31023450001
NFO 174140.07855125
EEB 436926.19334100006
RXI 2024576.75252
DBV 901160.5675925
PRFZ 3865639.549507
VOE 31357405.837466005
VOT 14410296.93337
OIL 15067871.6734
GSG 5102737.012936
QID 36868813.116784
DXD 14791669.552718
MZZ 166179.80198999998
SDS 122486305.095693
FTCS 2287929.1297660596
FXA 2186764.79398
FXS 141966.1614
FXC 4972783.772473
FXF 1725662.393622
FXB 13261092.805947
FDN 41699984.713942
FBT 7426043.94152
XPH 7121360.406884
XES 8846558.9769
XOP 524354035.416497
KRE 381765324.31197596
XME 124839994.870704
XRT 205152532.141776
DDM 19090612.802775
PSQ 17586748.325875
MYY 324838.739001
MVV 4280416.423724
SSO 228816529.186536
DOG 10383352.646586
SH 78213357.25979601
QLD 112513667.178186
DNL 455837.38022349996
DXJ 210492871.938966
DOO 982082.12965
DFE 6916435.832996999
DTD 1845328.0304999999
DWM 3211348.611045
DES 8735880.515088001
DIM 921262.7104239999
DTH 9

In [3]:
##ADD ALL ETFS
for item in etfDailyTradingValue:
    cloud.addETFVolume(item, etfDailyTradingValue[item])

In [3]:
##GET VALID ETFs
ONE_MILLION = 1000000
TEN_MILLION = ONE_MILLION * 10
ONE_BILLION = ONE_MILLION * 1000
TEN_BILLION = ONE_BILLION * 10


In [12]:
for tick in cloud.getValidETFVolumes(ONE_MILLION * 100):
    print(tick, cloud.getName(tick))

EWG iShares MSCI Germany Index Fund
FEZ SPDR EURO STOXX 50 ETF
IWO iShares Russell 2000 Growth Index Fund
QLD ProShares Ultra QQQ
SHY iShares 1-3 Year Treasury Bond ETF
SDS ProShares UltraShort S&P500
XME SPDR S&P Metals & Mining ETF
SLV iShares Silver Trust
VTV Vanguard Value ETF
KBE SPDR S&P Bank ETF
IWB iShares Russell 1000 Index Fund
OIH Market Vectors Oil Service ETF
IWN iShares Russell 2000 Value Index Fund
EWT iShares MSCI Taiwan Capped ETF
EWY iShares MSCI South Korea Index Fund
TIP iShares Barclays TIPS Bond Fund
IJR iShares S&P SmallCap 600 Index Fund
XRT SPDR S&P Retail ETF
IEF iShares 7-10 Year Treasury Bond ETF
IWF iShares Russell 1000 Growth Index Fund
DXJ WisdomTree Japan Hedged Equity Fund
EWW iShares MSCI Mexico Investable Market Index Fund
VGK Vanguard MSCI European ETF
SMH Market Vectors Semiconductor ETF
SSO ProShares Ultra S&P500
IJH iShares S&P MidCap 400 Index Fund
XLB Materials Select Sector SPDR Fund
IWD iShares Russell 1000 Value Index Fund
VTI Vanguard Total 

In [15]:
for tick in cloud.getValidETFVolumes(ONE_MILLION * 50):
    print(tick, cloud.getName(tick))

IWR iShares Russell Midcap Index Fund
IYF iShares Dow Jones US Financial Sector Index Fund
VBR Vanguard Small-Cap Value ETF
ILF iShares S&P Latin America 40 Index Fund
OEF iShares S&P 100 Index Fund
IYT iShares Transportation Average ETF
VFH Vanguard Financials ETF
SDY SPDR S&P Dividend ETF
VO Vanguard Mid-Cap ETF
VIG Vanguard Dividend Appreciation ETF
VGT Vanguard Information Technology ETF
XHB SPDR S&P Homebuilders ETF
EWC iShares MSCI Canada Index Fund
FXE CurrencyShares Euro Trust
EWH iShares MSCI Hong Kong Index Fund
SOXX iShares PHLX Semiconductor ETF
DVY iShares Select Dividend ETF
SH ProShares Short S&P500
EWU iShares MSCI United Kingdom Index Fund
RSP Guggenheim S&P 500 Equal Weight ETF
ITB iShares U.S. Home Construction ETF
IAU iShares Gold Trust
VB Vanguard Small-Cap ETF
IVW iShares S&P 500 Growth Index Fund
VUG Vanguard Growth ETF
IVE iShares S&P 500 Value Index Fund
EWG iShares MSCI Germany Index Fund
FEZ SPDR EURO STOXX 50 ETF
IWO iShares Russell 2000 Growth Index Fund
QL

In [4]:
cachedData = {}

In [5]:
##ADD DATASOURCES
import empyrical
tradeableETFs = cloud.getValidETFVolumes(ONE_MILLION * 50)
toUpload = []
for tick in tradeableETFs:
    name = cloud.getName(tick).lower()
    print("OBSERVING", tick, name)
    if 'ultra' in name or 'short' in name:
        print("SKIPPING", tick, name)
        continue
    thisData = backtestSimulator.formulateReturnsForTicker(tick, DA.getTickerData(tick)) if tick not in cachedData else cachedData[tick]
    cachedData[tick] = thisData
    for subtick in tradeableETFs:
        if subtick == tick:
            continue
        subData = backtestSimulator.formulateReturnsForTicker(subtick, DA.getTickerData(subtick)) if subtick not in cachedData else cachedData[subtick]
        cachedData[subtick] = subData
        bt = empyrical.alpha_beta(subData[:"2016-01-01"], thisData[:"2016-01-01"])[1]
        if abs(bt) < 0.85:
            print(tick, subtick, abs(bt))
            ##UPLOAD
            toUpload.append((tick, subtick))

OBSERVING IWR ishares russell midcap index fund
IWR OEF 0.848157240775
IWR VIG 0.760507154768
IWR FXE 0.137150348218
IWR IAU 0.0705966683381
IWR IVW 0.832177576747
IWR SHY 0.0244710880574
IWR SLV 0.380253149397
IWR TIP 0.0596240961799
IWR IEF 0.133566419812
IWR DXJ 0.70502873583
IWR AGG 0.0307684941884
IWR USO 0.671678127692
IWR EWJ 0.759806209297
IWR LQD 0.0199000822909
IWR XLP 0.487009987509
IWR XLV 0.638530050539
IWR DIA 0.793587340625
IWR XLU 0.609597937275
IWR GLD 0.0749817676285
IWR TLT 0.27433295755
IWR GDX 0.699890601276
OBSERVING IYF ishares dow jones us financial sector index fund
IYF IWR 0.622564071645
IYF VBR 0.720952771481
IYF ILF 0.750739366755
IYF OEF 0.590989127653
IYF IYT 0.637250696703
IYF SDY 0.599297168055
IYF VO 0.634102850579
IYF VIG 0.478360475493
IYF VGT 0.535116300452
IYF EWC 0.491388933764
IYF FXE 0.0728106068957
IYF EWH 0.574621873898
IYF SOXX 0.709455013583
IYF DVY 0.599506940171
IYF SH 0.560985157498
IYF EWU 0.560974637826
IYF RSP 0.648611133769
IYF IAU 0.0

In [17]:
cloud.addDataSourcesForTicker(toUpload)